##  Factorization Machine
###  Intro
* FM model is very efficient in fitting dataset with sparse matrix.
* It was applied in Kaggle competition for Click-Through-Rate(CTR) prediction, and achieved excellent results.
* FM model has the following strengths:
    * Works with very sparse data where SVMs fail.
    * By introducing interaction terms to linear models,it generates higher dimensional features and improves accuracy compared to simple logistic models
    * FM has linear complexity,and do not reply on support vectors like SVMs
* Other applications:
    * Regression
    * Binary Classification
    * Ranking
    
### Mathematics
* target function with interaction terms

<img src="files/fm_formula.png">
<img src="files/fm_vector.png">
* derivation for linear complexity
<img src="files/fm_inter.png">

Loss functions
* regression(MSE)
<img src="files/fm_rg_func.png">
<img src="files/fm_rg_func2.png">

* classification(Logit loss)
<img src="files/fm_lg_func.png">
<img src="files/fm_lg_func2.png">
<img src="files/fm_lg_func3.png">

* Gradient for classificaiton with sigmoid loss
<img src="files/fm_loss_func.png">

* The algo goes
<img src="files/fm_sgd.png">

#### References
* Rendle, Factorization Machines.
* Factorization Machines with libFM
* FM:http://www.cs.cmu.edu/~wcohen/10-605/2015-guest-lecture/FM.pdf 
* A quick summary on FM and FFM from meituan:https://tech.meituan.com/2016/03/03/deep-understanding-of-ffm-principles-and-practices.html
* Kaggle competition:https://www.kaggle.com/c/avazu-ctr-prediction/discussion/10729#latest-236361
* https://www.cnblogs.com/wkang/p/9588360.html#4267887
* https://blog.csdn.net/google19890102/article/details/45532745

Below is an implementation example for FM in python

In [136]:
from math import exp
from numpy import *
from random import normalvariate # random number from normal distribution
from datetime import datetime
from sklearn import datasets

def sigmoid(x): return 1.0/(1+exp(-x))

class FactorizationMachine():
    """
    example to fit factorization machine using stochastic gradient descent 
    """
    def __init__(self,k=5,alpha=0.01,max_iter=10,verbose=True):
        self.max_iter = max_iter
        self.k = k  # dimension(columns) for factor v
        self.alpha = alpha # learning rate
        self._verbose = verbose
    
    def predict_proba(self,test_X):
        y_prob = []
        # for easy calc
        v = self.v
        w_0 = self.w_0
        w = self.w
        for row_i in range(test_X.shape[0]):
            inter1 = test_X[row_i] * v
            inter2 = multiply(test_X[row_i],test_X[row_i]) * multiply(v,v)
            interaction = sum(multiply(inter1,inter1) - inter2)/2.
            p = w_0 + test_X[row_i] * w + interaction
            prob = sigmoid(p[0,0])
            y_prob.append(prob)
        return y_prob
    
    def predict(self,test_X):
        y_pred = []
        # for easy calc
        v = self.v
        w_0 = self.w_0
        w = self.w
        for row_i in range(test_X.shape[0]):
            inter1 = mat(test_X[row_i]) * v    # x_i * v_i
            inter2 = mat(multiply(test_X[row_i],test_X[row_i])) * multiply(v,v)
            interaction = sum(multiply(inter1,inter1) - inter2)/2.
            p = w_0 + test_X[row_i] * w + interaction
            prob = sigmoid(p[0,0])
            y_pred.append(1 if prob>.5 else 0)
        return y_pred
        
    def _SGDSolver(self):
        # for easy calc
        v = self.v
        w_0 = self.w_0
        w = self.w
        
        for itr in range(self.max_iter):
            if self._verbose:
                print('iter  number: ',itr)
            
            for row_i in range(self.nrows):  # n samples
                inter1 = self.train_X[row_i] * v                
                inter2 = multiply(self.train_X[row_i],self.train_X[row_i]) * multiply(v,v)
                interaction = sum(multiply(inter1,inter1) - inter2)/2.
                p = w_0 + transpose(self.train_X[row_i]) * w + interaction

                loss = sigmoid(self.train_y[row_i]*p[0,0]) - 1 # derivative  of sigmoid 
                if self._verbose:
                    print('sample no: ',row_i,'pred: ',p,' loss: ', self.alpha * loss * self.train_y[row_i])
                    
                w_0 = w_0 - self.alpha * loss * self.train_y[row_i]
                
                # update w
                for i in range(self.ncols):
                    if self.train_X[row_i,i] != 0:
                        w[i,0] = w[i,0] -self.alpha * loss * self.train_y[row_i] * self.train_X[row_i,i]
                
                # update v
                for i in range(self.ncols):
                    for j in range(self.k):
                        v[i,j] = v[i,j] - self.alpha * loss * self.train_y[row_i] * (self.train_X[row_i,i] * inter1[0,j] - v[i,j] * self.train_X[row_i,i] * self.train_X[row_i,i])
    
        # assign results
        self.v = v
        self.w_0 = w_0
        self.w = w
    
    def fit(self,train_X,train_y):
        self.train_X = train_X
        self.train_y = train_y
        self.nrows,self.ncols = shape(train_X)
        # initialize factors
        self.w_0 = 0
        self.w = mat(zeros((self.ncols,1)))
        self.v = mat(normalvariate(0,.2) * ones((self.ncols,self.k)))
        if self._verbose:
            print(self.train_X.shape,',',self.w.shape,',',self.v.shape)
        # run solver algo
        self._SGDSolver() 

test

In [140]:
from sklearn.datasets import make_classification,make_blobs
from sklearn.model_selection import train_test_split
X,y = make_blobs(n_samples=1000,random_state=1,n_features=10)
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=300)

In [ ]:
fm = FactorizationMachine(max_iter=100,k=8,verbose=False)
fm.fit(train_X,train_y)

In [ ]:
y_pred = fm.predict(train_X)
(train_y==y_pred).astype(int).sum()/len(train_y)
y_pred = fm.predict(test_X)
(test_y==y_pred).astype(int).sum()/len(test_y)